In [18]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
   ---------------------------------------- 0.0/165.1 kB ? eta -:--:--
   -------------- ------------------------- 61.4/165.1 kB 1.7 MB/s eta 0:00:01
   ----------------------------- ---------- 122.9/165.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 165.1/165.1 kB 1.4 MB/s eta 0:00:00
Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
   ---------------------------------------- 0.0/669.8 kB ? eta -:--:--
   --- ------------------------------------ 61.4/669.8 kB 3.2 MB/s eta 0:00:01
   -------- ------------------------------- 143.4/669.8 kB 2.1 MB/s eta 0:00:01
   -------------- ------------------------- 245.8/669.8 kB 2.1 MB/s eta 0:00:01
   -------------------- ------------------- 337.9/669.8 kB 2.1 MB/s eta 0:00:01
   -------------------------- ------------- 440.3/669.8 kB 2.0 MB/s eta 0:00:01
   ------------------------------- -------- 532.5/669.8 kB 2.0 MB/s eta 0:00:01
 

In [21]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Request hourly weather variables only (no daily aggregation)
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 40.7143,
    "longitude": -74.006,
    "start_date": "2023-01-01",
    "end_date": "2023-12-31",
    "hourly": [
        "temperature_2m",
        "wind_speed_10m",
        "rain",
        "snowfall",
        "precipitation",
        "wind_speed_100m"
    ],
    "timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    ),
    "temperature_2m": hourly_temperature_2m,
    "wind_speed_10m": hourly_wind_speed_10m,
    "rain": hourly_rain,
    "snowfall": hourly_snowfall,
    "precipitation": hourly_precipitation,
    "wind_speed_100m": hourly_wind_speed_100m
}

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Save the hourly data to a CSV file in data/processed
import os
output_dir = os.path.join('..', 'data', 'processed')
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'weather_hourly.csv')
hourly_dataframe.to_csv(output_path, index=False)
print(f"✅ Data saved to {output_path}")

Coordinates 40.738136291503906°N -74.04254150390625°E
Elevation 51.0 m asl
Timezone b'America/New_York'b'GMT-4'
Timezone difference to GMT+0 -14400 s
                          date  temperature_2m  wind_speed_10m  rain  \
0    2023-01-01 04:00:00+00:00         10.6225       11.298495   1.0   
1    2023-01-01 05:00:00+00:00         10.3725        7.421590   1.0   
2    2023-01-01 06:00:00+00:00         10.3225       13.104198   0.1   
3    2023-01-01 07:00:00+00:00         10.2225       15.986595   0.0   
4    2023-01-01 08:00:00+00:00          9.5725       16.039202   0.0   
...                        ...             ...             ...   ...   
8755 2023-12-31 23:00:00+00:00          3.3225        7.386582   0.0   
8756 2024-01-01 00:00:00+00:00          3.5225        5.692099   0.0   
8757 2024-01-01 01:00:00+00:00          3.5225        5.959060   0.0   
8758 2024-01-01 02:00:00+00:00          2.8225        6.792466   0.0   
8759 2024-01-01 03:00:00+00:00          2.3225        8.37